# MultiplEYE preprocessing

In [1]:
from pathlib import Path

data_dir = Path("../data/pilot-hr-1-zh")
stim_dir = Path("../data/stimuli_MultiplEYE_HR_CH_Zurich_1_2025")
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

## EDF to ASC

Use the `edf2asc` binary from the [EyeLink Developers Kit](https://www.sr-research.com/support/thread-13.html) to convert EDF files to ASC files.

To discuss:
- We probably can't distribute the binary due to licensing issues. (But we might be able to distribute a Docker image?)
- There is already an [issue](https://github.com/aeye-lab/pymovements/issues/509) to integrate this into `pymovements`.
- The `-input` option is unnecessary, but currently required by `parse_eyelink()` in `pymovements`.

In [2]:
import subprocess

edf = data_dir / "ch1hr007.edf"
subprocess.run(["./edf2asc", edf, "-input", "-p", output_dir, "-y"])


EDF2ASC: EyeLink EDF file -> ASCII (text) file translator
EDF2ASC version 4.2.762.0 Linux   standalone Jul 20 2023 
(c)1995-2023 by SR Research, last modified Jul 20 2023

processing file ../data/pilot-hr-1-zh/ch1hr007.edf 
loadEvents = 1
==============Preamble of file ../data/pilot-hr-1-zh/ch1hr007.edf==============
| DATE: Wed Jun 19 17:49:30 2024                                              |
| TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED                                   |
| VERSION: EYELINK II 1                                                       |
| SOURCE: EYELINK CL                                                          |
| EYELINK II CL v6.14 Mar  6 2020 (EyeLink Portable Duo)                      |
| CAMERA: EyeLink USBCAM Version 1.01                                         |
| SERIAL NUMBER: CLU-DBE08                                                    |
| CAMERA_CONFIG: DBE08200.SCD                                                 |
| RECORDED BY libeyelink.py             

CompletedProcess(args=['./edf2asc', PosixPath('../data/pilot-hr-1-zh/ch1hr007.edf'), '-input', '-p', PosixPath('output'), '-y'], returncode=255)

ample at 2675320.500000 
missing sample at 2689794.500000 
missing sample at 2692863.500000 
missing sample at 2720722.500000 
missing sample at 2822118.500000 
missing sample at 3045886.500000 
missing sample at 3106620.500000 
missing sample at 3188916.500000 
missing sample at 3206301.500000 
missing sample at 3391546.500000 
missing sample at 3420093.500000 
missing sample at 3426127.500000 
missing sample at 3578011.500000 
missing sample at 3602479.500000 
missing sample at 3645916.500000 
missing sample at 3664998.500000 
missing sample at 3683847.500000 
missing sample at 3787312.500000 
missing sample at 3818034.500000 
missing sample at 3859301.500000 
missing sample at 3864391.500000 
missing sample at 3905822.500000 
missing sample at 3981997.500000 
Converted successfully: 26935 events, 3220492 samples, 404 blocks.


## ASC to sample CSV

Convert the ASC files to a CSV file where each row is a sample.

TODO:
- Import experiment config (screen size etc.) from stimulus directory.
- Split the CSV files into separate files for each trial/screen.
- Extract metadata (not yet available through `from_asc()`).

In [3]:
import pymovements as pm
import polars as pl

asc = output_dir / "ch1hr007.asc"

experiment = pm.Experiment(
    sampling_rate=1000,
    screen_width_px=1275,
    screen_height_px=916,
    screen_width_cm=37,
    screen_height_cm=28,
    distance_cm=60,
)
data = pm.gaze.from_asc(
    asc,
    patterns=[
        r"start_recording_(?P<trial>(?:PRACTICE_)?trial_\d+)_(?P<screen>.+)",
        {"pattern": r"stop_recording_", "column": "trial", "value": None},
        {"pattern": r"stop_recording_", "column": "screen", "value": None},
        {
            "pattern": r"start_recording_(?:PRACTICE_)?trial_\d+_page_\d+",
            "column": "activity",
            "value": "reading",
        },
        {
            "pattern": r"start_recording_(?:PRACTICE_)?trial_\d+_question_\d+",
            "column": "activity",
            "value": "question",
        },
        {
            "pattern": r"start_recording_(?:PRACTICE_)?trial_\d+_(familiarity_rating_screen_\d+|subject_difficulty_screen)",
            "column": "activity",
            "value": "rating",
        },
        {"pattern": r"stop_recording_", "column": "activity", "value": None},
        {
            "pattern": r"start_recording_PRACTICE_trial_",
            "column": "practice",
            "value": True,
        },
        {
            "pattern": r"start_recording_trial_",
            "column": "practice",
            "value": False,
        },
        {"pattern": r"stop_recording_", "column": "practice", "value": None},
    ],
    # trial_columns=["trial", "screen"],
    experiment=experiment,
)

# Convert [x, y] pixel column to separate pixel_x and piyel_y columns
df = data.frame.select(
    [
        pl.all().exclude("pixel"),
        pl.col("pixel").list.get(0).alias("pixel_x"),
        pl.col("pixel").list.get(1).alias("pixel_y"),
    ]
)

df.write_csv(output_dir / f"samples_ch1hr007.csv")

/home/andreas/work/uzh/multipleye/multipleye-preprocessing/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
